In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [3]:
def language_count(garbage):
    top_tier_languages = {'Python','C', 'C++', 'JavaScript', 'TypeScript', 'Ruby', 'Rust', 'Lua', 'Assembly', 'Zig'}
    return len(set(garbage.replace(' ', '').replace('{', '').replace('}','').replace("\'", '').split(',')) & top_tier_languages)


In [4]:
unlabeled_data =  pd.read_csv('resources/labeled_data.csv')
unlabeled_data = np.array(unlabeled_data)[:15000]
print(unlabeled_data.shape)

(15000, 12)


In [29]:
processed_unlabeled_data = np.zeros((15000, 6), dtype=int)
processed_unlabeled_data[:, :4] = unlabeled_data[:, (4,6,7,11)].astype(int)
processed_unlabeled_data[:, -2] = np.array([language_count(langs) for langs in unlabeled_data[:, 9]])
processed_unlabeled_data[:, -1]

array([0, 0, 0, ..., 0, 0, 0])

In [42]:
# First batch
scaler = StandardScaler() 
first_batch = scaler.fit_transform(processed_unlabeled_data[:8000, :5])
first_batch.shape

# Second batch
second_batch = scaler.fit_transform(processed_unlabeled_data[8000:, :5])
second_batch.shape


(7000, 5)

In [8]:
# @title Neural Network
class Layer_Dense:
  def __init__(self,n_inputs,n_neurons):
    self.weights = 0.01 * np.random.randn(n_inputs,n_neurons)
    self.biases = np.zeros((1,n_neurons))

  def forward(self,inputs):
    self.output = np.dot(inputs, self.weights)+ self.biases

  def parameters(self):
    return self.weights

class Activation_ReLU:
  def forward(self,inputs):
    self.output = np.maximum(0,inputs)

class Activation_Softmax:
  def forward(self,inputs):
    exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
    probabilities = exp_values/np.sum(exp_values,axis=1,keepdims=True)
    self.output = probabilities


class Loss_Categoricalcrossentropy():
  def forward(self, probs, y_true):
    samples = len(probs)
    probs_clipped = np.clip(probs, 1e-7, 1 - 1e-7)
    #negative_log_likelihoods = -np.sum(y_true*np.log(probs_clipped))
    #final_loss = np.sum(negative_log_likelihoods)/samples
    correct_confidences = np.sum(probs_clipped * y_true, axis=1)
    n_l_l = -np.log(correct_confidences)
    final_loss = np.mean(n_l_l)
    return final_loss


In [17]:
layer1 = Layer_Dense(5,32)
layer2 = Layer_Dense(32,2)
activation1 = Activation_ReLU()
activation2 = Activation_Softmax()

In [18]:
weights = np.load('resources/weights.npy', allow_pickle=True)
layer1.weights = weights.item().get('w1')
layer2.weights = weights.item().get('w2')

layer1.biases = weights.item().get('b1')
layer2.biases = weights.item().get('b2')


In [43]:
layer1.forward(second_batch)
activation1.forward(layer1.output)
a1 = activation1.output
print(a1.shape)
layer2.forward(activation1.output)
activation2.forward(layer2.output)
logits = activation2.output
print(logits.shape)

(7000, 32)
(7000, 2)


In [44]:
y_pred = np.argmax(activation2.output,axis=1)
processed_unlabeled_data[8000:, -1] = y_pred

In [48]:
processed_unlabeled_data[:, -1]
X = processed_unlabeled_data[:, :]
X.shape
np.save('resources/nninput.npy', X)